# Standardisation des données et autres transformations
Standardiser ou normaliser les données ne sont au final que des cas particulier de transformations (monotones) des données permettant de garantir ou d'améliorer les performances des algorithmes situés en aval.

Remarque : Les transformations monotones des *features* conservent par définition l'ordre des observations suivant la *feature* transformée. En dehors de la normalisation ou de la standardisation on peut citer la *log transform* des *features*, ou encore la *power* ou *quantile transform*, toutes trois ayant comme effet (ce qui peut être recherché) de modifier la distribution initiale des données pour la *feature* considérée. Tous les modèles n'y sont pas sensibles.

On fait la distinction entre standardisation et normalisation:
* **La standardisation** correspond à un centrage et une réduction (*rescaling*) des données pour obtenir un échantillon de moyenne nulle et d'écart-type $1$ : $x_i \leftarrow \frac{x_i - \bar{x}}{\sigma}$ On peut définir la standardisation comme le procédé mettant à la même échelle l'ensemble des *features*.
* **La normalisation** correspond à un *mapping* des données vers un intervalle borné, en général $[0,1]$, à l'aide de la transformation suivante $x_i \leftarrow \frac{x_i - x_{min}}{x_{max} - x_{min}}$ Exemple d'application: Canaux RGB d'une image compris entre 0 et 255 ramenés dans $[0, 1]$, plage de données préférée par les *neural networks*.

Remarques:
* Normalisation et standardisation n'ont pas le même comportement face aux *outliers*.
* La standardisation de l'ensemble des *features* et associée à une légère perte d'information: celle des variances relatives entre *features* (elles sont finalement toutes de variance $1$). Cette étape est cependant indispensable dans le cas de *features* aux échelles très différentes (cf. plus bas).
* Après standardisation, la matrice de covariance devient matrice de corrélation. D'après le point précédent, elles ne contiennent pas rigoureusement la même information.

Quelle que soit la technique utilisée, on doit se poser les questions:
* La transformation préserve-t-elle la forme de la distribution ?
* La transformation est-elle sensible à la présence d'*outliers* ?
* Vers quel intervalle la transformation *map*-t-elle les données ?

Ces trois questions pouvant être liées. Par exemple: Une normalisation va faire passer une distribution de données d'un support potentiellement infini à un support fini avec une distortion d'autant plus importante que des *outliers* sont présents. On a alors un risque de distordre les relations avec d'autres *features* *scalées* de la même façon mais sans *outliers* (notre première distribution sera anormalement ressérrée comparée aux autres).

## Pourquoi *scaler* / standardiser ses données ?
Standardiser les données est souvent indispensable quand les échelle des différentes *features* sont potentiellement très différentes (ex: une *feature* prend ses valeurs dans $[0, 1]$ quand une autre les prend dans $[0, 10000]$ et que l'algorithme utilisé par la suite est sensible à ces différences d'échelle (en général en se biaisant / en surpondérant l'influence des *features* prenant les plus fortes valeurs). On peut citer comme exemples:
* Si l'algorithme repose sur des calculs de distances par exemple euclidiennes (k-NN, k-means, etc.): Les features aux larges valeurs auront une influence disproportionnée sur la valeur prise par la distance. Marche plus largement pour des mesures de similarité et les algorithmes se basant dessus.
* La PCA qui consiste à rechercher les directions de variance maximale. L'expression de la variance faisant appel à une notion de distance, on retrouve le problème décrit au point précédent: les *features* prenant leurs valeurs sur une large plage auront une contribution disproportionnée à la variance.

D'autres algorithmes sont par design immunisés à ce genre d'effet, on peut citer les arbres de décision CART (ils sont sans doute les seuls) dont le critère de *split* utilisé pour leur croissance n'est calculé que sur l'ordre des observations pour la *feature* choisie pour le *split*. Il sont donc ainsi immunisés aux données non normalisées mais aussi en partie à la présence d'*outliers* (qui peuvent tout de même les faire *overfitter*). Tous les modèles dérivés des arbres CART bénéficient également de cette propriété: *random forest* et *gradient boosting* notamment.

La standardisation permet aussi une meilleure estimation et stabilité de l'estimation des coefficients du modèle. Dans le cas des algorithmes consistant à descendre une fonction de coût (régression linéaire et logistique, *neural networks*, etc.), le fait d'avoir des échelles très différentes peut faire que certains poids peuvent s'updater beaucoup plus vite que d'autres (les *features* $x_{ij}$ intervenant dans l'expression d'update des coefficients) pouvant dans le pire des cas, nous empêcher de converger vers l'optimum voire nous le faisant rater (exemple: de la fonction de coût en forme de bol très étiré).

Remarque: Ce raisonnement semble s'appliquer aux algorithmes maximisant une vraisemblance (ex: régression logistique). Là encore, cela dépend de la méthode choisie.

Certains algorithmes exigent que les *features* qui leur sont présentées prennent leurs valeurs dans un certain intervalle rendant la standardisation indispensable. D'autres exigent (par hypothèse) que la distribution des *features* d'entrée suive une certaine forme (gaussienne notamment), on utilise alors d'autres techniques dédiées (*quantile* et *power transform* notamment).

Remarques:
* Sans standardiser et avec des *features* prenant de très grandes valeurs, on risque de se retrouver avec des coefficients/poids avec de très faibles valeurs (ex: $10^{-6}$) ceux-ci intégrant ces différences d'échelle. Cela pose plusieurs problèmes: fiabilité et stabilité de l'estimation, approximations et erreurs lors du calcul, régularisation (?), etc.
* En présence de régularisation (type Ridge ou LASSO), avoir des variables d'échelles très différentes peut sembler contradictoire:
     * Comment contraindre efficacement la norme du vecteur de coefficients si les coefficients associés à certaines *features* vont y prendre une importance disproportionnellement faible ou élevée du fait de leur unité ?
     * Comment sélectionner les *features* en écartant celles aux plus faibles coefficients si certains d'entre eux sont relativement plus faibles du simple fait de les échelles sur lesquelles leurs *features* prennent leurs valeurs sont relativement plus élevées ?

### Conclusion
**De manière générale et par prudence, au minimum standardiser ses données**, ça ne peut pas faire beaucoup de mal. Toutes les techiques de transformation mentionnées ici sont à retrouver dans `sklearn.preprocessing`.